In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing
import dask.array as da
import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder
from dask_ml.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:63584 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [2]:
cars_df = dd.read_csv('/Users/steveangeli/Downloads/vehicles.csv')

In [3]:
cars_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 25 entries, id to long
dtypes: object(18), float64(5), int64(2)

In [4]:
cars_df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7034441763,https://saltlakecity.craigslist.org/cto/d/salt...,salt lake city,https://saltlakecity.craigslist.org,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,...,4wd,compact,hatchback,black,https://images.craigslist.org/00G0G_fTLDWM5Xyv...,PRICE REDUCED! -Garage kept -Low Miles (63K)...,NaN,ut,40.7372,-111.858
1,7034440610,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00v0v_7Cu0buIofU...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
2,7034440588,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,46463,2015.0,gmc,sierra 1500,excellent,NaN,...,4wd,NaN,NaN,white,https://images.craigslist.org/01515_lPvJ9bfbdY...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
3,7034440546,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_6Rjfp3NS4O...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
4,7034406932,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,49999,2018.0,ford,f-450,NaN,NaN,...,4wd,NaN,pickup,white,https://images.craigslist.org/00W0W_8yIUwRBXXd...,2018 Ford F-350 F350 F 350 SD Lariat Crew Cab ...,NaN,ut,40.3744,-104.694


In [5]:
# Drop some columns that I don't believe wil have much impact on the sale price or that have big problems.
cars_df = cars_df.drop(columns=['url', 'model', 'region_url', 'vin', 'image_url', 'description', 'county', 'lat', 'long', 'id', 'region', 'state'])

In [6]:
# Drop any null rows because I still should have plenty of data to work with. 
cars_df = cars_df.dropna()
cars_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 13 entries, price to paint_color
dtypes: object(10), float64(2), int64(1)

In [7]:
non_numeric_columns = cars_df.select_dtypes(['object']).columns
numeric_columns = cars_df.select_dtypes(['int64', 'float64']).columns

In [8]:
cars_df = cars_df.categorize(columns=non_numeric_columns)
de = DummyEncoder()
trn_cars_df = de.fit_transform(cars_df)
trn_cars_df.head()

,price,year,odometer,manufacturer_volkswagen,manufacturer_honda,manufacturer_ford,manufacturer_nissan,manufacturer_jeep,manufacturer_gmc,manufacturer_dodge,...,paint_color_white,paint_color_blue,paint_color_custom,paint_color_yellow,paint_color_silver,paint_color_red,paint_color_green,paint_color_brown,paint_color_purple,paint_color_orange
0,17899,2012.0,63500.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,4600,2008.0,110982.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,28000,2004.0,67348.0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
57,2500,2004.0,129000.0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
109,12000,2015.0,65000.0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
Y = da.log(trn_cars_df['price'])
X = trn_cars_df.drop(['price'], axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.

In [ ]:
lr = LinearRegression()

with joblib.parallel_backend('dask'):
    lr.fit(X_train.values.compute(), y_train.values.compute())

    preds_train = lr.predict(X_train.values.compute())
    preds_test = lr.predict(X_test.values.compute())

    print("R-squared of the model in training set is: {}".format(lr.score(X_train.values.compute(), y_train.values.compute())))
    print("R-squared of the model on the test set is: {}".format(lr.score(X_test.values.compute(), y_test.values.compute())))